In [1]:
using Distributions, CSV, DataFrames, ProgressMeter
include("../../utils_1d.jl")

MethodOfLines (generic function with 1 method)

In [2]:
T=1.0; r=0.01; R = 0.3; μ=0.03; S0 = 100.0; σ = 0.2; K = 100.0;

drift(x) = μ*x
diffusion(x) = σ*x
driver(t, x, y, z) = (
        -r*max(y-z/σ, 0.0)
        -R*min(y-z/σ, 0.0)
        -μ*z/σ
        )
# terminal(x) = max(K-x[1],0) 
terminal(x) = max(x-K,0)

bsde = BSDE(T, S0, drift, diffusion, driver, terminal);

In [3]:
Nₜs = [10, 20, 50, 100, 200]
header = vcat(["scheme", "measurement_type"], string.(Nₜs))
schemes = [
    [LawsonEuler(krylov=true, m=100), true],
    [NorsettEuler(krylov=true, m=100), true],
    [ETDRK2(krylov=true, m=100), true],
    [ETDRK3(krylov=true, m=100), true],
    [ETDRK4(krylov=true, m=100), true],
    [HochOst4(krylov=true, m=100), true]
    # [DP5(), false],
    # [RadauIIA5(), false]
]

6-element Vector{Vector{Any}}:
 [LawsonEuler{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [NorsettEuler{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [ETDRK2{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [ETDRK3{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [ETDRK4{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [HochOst4{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]

In [4]:
### designing grids
domain = [0.0, 2*bsde.X0];
Nₗ = 1000; Δₗ = (bsde.X0-domain[1])/Nₗ;
Nᵣ = 1000; Δᵣ = (domain[2]-bsde.X0)/Nᵣ;

g₁ = 50.0; g₂ = 50.0
spatial_grid = TavellaRandallGrid(g₁, g₂, domain[1], bsde.X0, domain[2], Nₗ, Nᵣ)

TavellaRandallGrid([0.0, 0.16129920473716197, 0.3223903372606429, 0.48327373329789225, 0.6439497281433404, 0.8044186566592799, 0.964680853276306, 1.1247366519943256, 1.2845863863829976, 1.4442303895825717  …  198.55576961041743, 198.715413613617, 198.87526334800566, 199.03531914672368, 199.19558134334073, 199.35605027185665, 199.5167262667021, 199.67760966273937, 199.83870079526284, 200.0], 2001, 1000, 1000)

In [5]:
d=2
df = DataFrame([[],[],[],[],[],[],[]], header)
@showprogress for attr in schemes
    abs100 = zeros(Float64, length(Nₜs)); 
    runtimes = zeros(Float64, length(Nₜs))
    scheme = attr[1]
    EXPINT = attr[2]
    @showprogress for (ind, Nₜ) in enumerate(Nₜs)
        print(string("Nₜ=", Nₜ,":\n")); flush(stdout)
        exc_start = time()
        res = MethodOfLines(bsde, spatial_grid, Nₜ, scheme, EXPINT)
        exc_stop = time()
        sol = res[1]; s_grid = res[2]; 
        
        index = spatial_grid.Nₗ + 1
        abs100[ind] = sol[index,end]
        runtimes[ind] = exc_stop - exc_start
        print(string("Y0(100)=", abs100[ind],":\n")); flush(stdout)
        print(string("Runtime[s]=", runtimes[ind],":\n")); flush(stdout)
    end
    schemename = split(split(string(scheme), '{')[1], '(')[1]
    row_100 = vcat([schemename, "Y0(100)"], string.(abs100)); push!(df,row_100);
    row_run = vcat([schemename, "Runtime[s]"], string.(runtimes)); push!(df,row_run);
end
df |> CSV.write(string("result_1108.csv"))

Nₜ=10:
Y0(100)=25.037743070245266:
Runtime[s]=5.256757020950317:
Nₜ=20:
Y0(100)=26.352226213278275:
Runtime[s]=0.6053400039672852:


Progress:  40%|████████████████▍                        |  ETA: 0:00:09

Nₜ=50:
Y0(100)=26.354533665706075:
Runtime[s]=1.5152418613433838:


Progress:  60%|████████████████████████▋                |  ETA: 0:00:05

Nₜ=100:
Y0(100)=26.343740915814383:
Runtime[s]=2.811455011367798:


...

Nₜ=100:
Y0(100)=26.330524752433757:
Runtime[s]=14.27010989189148:


Progress:  80%|████████████████████████████████▊        |  ETA: 0:00:07

Nₜ=200:
Y0(100)=26.330528094426597:
Runtime[s]=29.638652086257935:


Progress: 100%|█████████████████████████████████████████| Time: 0:00:55
Progress: 100%|█████████████████████████████████████████| Time: 0:03:11


"result_1108.csv"